In [ ]:
%pip install langchain_community groq faiss-cpu sentence-transformers pypdf langchain_huggingface



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_core.documents import Document

# loader = PyPDFLoader("doc.pdf")
# docs = loader.load()

# len(docs)  

26

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
# chunks = text_splitter.split_documents(docs)
# len(chunks)  

56

In [ ]:
# documents = chunks
# documents

[Document(metadata={'producer': 'Aspose.PDF for .NET 19.8', 'creator': 'Aspose Ltd.', 'creationdate': '', 'moddate': '2023-04-06T10:11:19-04:00', 'source': 'doc.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1'}, page_content='UNITED STATES DISTRICT COURT \nNORTHERN DISTRICT OF TEXAS \nAMARILLO DIVISION \n \nALLIANCE FOR HIPPOCRATIC \nMEDICINE, et al., \n  \n   \n Plaintiffs,   \n   \n v.  Case No. 2:22-cv-00223-Z \n   \nU.S. FOOD AND DRUG \nADMINISTRATION, et al., \n  \n   \n Defendants. \n \n  \n \n \nAMICUS CURIAE BRIEF OF MISSISSIPPI, ALABAMA, ALASKA, \nARKANSAS, FLORIDA, GEORGIA, IDAHO, INDIANA, IOWA, KANSAS, \nKENTUCKY, LOUISIANA, MONTANA, NEBRASKA, OHIO, OKLAHOMA, \nSOUTH CAROLINA, SOUTH DAKOTA, TENNESSEE, TEXAS, UTAH, AND \nWYOMING IN SUPPORT OF PLAINTIFFS’ MOTION  \nFOR PRELIMINARY INJUNCTION \n \n \n \nCase 2:22-cv-00223-Z   Document 100   Filed 02/13/23    Page 1 of 26   PageID 3760'),
 Document(metadata={'producer': 'Aspose.PDF for .NET 19.8', 'creator': 'Aspose Ltd.', 

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
# embeddings = HuggingFaceEmbeddings()

# db = FAISS.from_documents(documents, embeddings)



In [ ]:
# db

In [ ]:
# query="Amici curiae are the States of Mississippi, Alabama, Alaska "
# result=db.similarity_search(query)
# result[0].page_content

'1 \n \nINTRODUCTION, INTEREST OF AMICI CURIAE,  \nAND SUMMARY OF ARGUMENT \nLast year, the Supreme Court held that abortion is a matter that is entrusted \nto “the people and their elected representatives” to address. Dobbs v. Jackson \nWomen’s Health Organization , 142 S. Ct. 2228, 2284 (2022). Overruling precedent \nthat took that auth ority away from the people, the Court returned the issue of \n“regulating or prohibiting abortion” to “the citizens of each State.” Ibid.  States may \nthus pursue their “legitimate interests” in protecting unborn life, women’s health, \nand the medical profession’s integrity by regulating or restricting abortion. Ibid. \nAmici curiae are the States of Mississippi, Alabama, Alaska, Arkansas, \nFlorida, Georgia, Idaho, Indiana, Iowa, Kansas, Kentucky, Louisiana, Montana, \nNebraska, Ohio, Oklahoma, South Carolina, South Dakota, Tennessee, Texas, Utah, \nand Wyoming. Like other States, amici have, consistent with the Constitution and'

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_template("""
# You are a legal analyst. Using ONLY the provided context, produce EXACTLY 10 pivotal items for the case.

# Structure the output as follows:
# -Concise Title
# -Arguments (supporting points for the chosen side, based only on the context)
# -Citations (precise page and line numbers from the context)

# Rules:

# Provide 5 items FOR the case and 5 items AGAINST the case.

# Do not mix both sides within one item — each item should belong to either the FOR group or the AGAINST group.

# Use only information in the provided context. Do not invent facts. If a point is implied but not explicit, note it briefly and cite the closest support.

# Output must contain exactly 10 items in total (5 FOR, 5 AGAINST).

# <context>
# {context}
# </context>
# Question: {input}""")

In [ ]:
# retriever=db.as_retriever()
# retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EC18B853C0>, search_kwargs={})

In [1]:
from groq import Groq
import os
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
print("Groq client initialized successfully!")


Groq client initialized successfully!


In [2]:
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, List, Optional
import json

class GroqLLM(LLM):
    """Custom Groq LLM wrapper for LangChain"""
    
    model_name: str = "llama-3.3-70b-versatile"
    temperature: float = 0.1
    max_tokens: int = 2048
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Call the Groq API"""
        try:
            response = client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                stop=stop,
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error calling Groq API: {str(e)}"
    
    @property
    def _llm_type(self) -> str:
        return "groq"


llm = GroqLLM(model_name="llama-3.3-70b-versatile", temperature=0.1)
print("Groq LLM initialized successfully!")


Groq LLM initialized successfully!


In [3]:
# Test query to verify LLM is working
test_response = llm("What is the capital of France?")
print("Test response:", test_response)


C:\Users\prani\AppData\Local\Temp\ipykernel_23420\1530287381.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_response = llm("What is the capital of France?")


Test response: The capital of France is Paris.


In [ ]:
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain

# # document chain with Groq LLM
# document_chain = create_stuff_documents_chain(llm, prompt)
# print("Document chain created with Groq LLM!")

# # retrieval chain with Groq LLM
# retrieval_chain = create_retrieval_chain(retriever, document_chain)
# print("Retrieval chain created with Groq LLM!")


Document chain created with Groq LLM!
Retrieval chain created with Groq LLM!


In [ ]:
# response = retrieval_chain.invoke({"input":"identify the top 10 key items that are pivotal to the case. These items must encompass both the arguments for and against, providing a balanced perspective that is crucial for strategic planning."})
# print("Groq-powered response:")
# print(response['answer'])


Groq-powered response:
**Pivotal Items in the Case Against the FDA's Actions**

### Arguments For the Case (Against FDA's Actions)
1. **Multiple State Support**: The case has the support of numerous state attorneys general, indicating a broad-based opposition to the FDA's actions (Page 6, Lines 19-24).
2. **Public Interest and Equities**: The conclusion states that the public interest and equities support relief against the FDA's actions, suggesting a strong argument in favor of the plaintiffs (Page 6, Line 18).
3. **Legal Precedent**: The case cites Dobbs v. Jackson Women’s Health Organization, which could imply a significant legal precedent supporting the plaintiffs' argument (Page 24, Line ii, citing Dobbs v. Jackson Women’s Health Organization).
4. **Federal Overreach**: The involvement of state attorneys general may imply an argument against federal overreach by the FDA, though this is not explicitly stated (Page 6, Lines 19-24).
5. **Constitutional Concerns**: The citation of cas

In [ ]:
# import pdfplumber
# from langchain_core.documents import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# from sentence_transformers import CrossEncoder   # ✅ reranker


# # ---- Step 1: Extract text with page + line numbers ----
# def extract_lines_from_pdf(pdf_path):
#     docs = []
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages, start=1):
#             if not page.extract_text():
#                 continue
#             lines = page.extract_text().split("\n")
#             for i, line in enumerate(lines, start=1):
#                 if line.strip():
#                     docs.append({
#                         "page": page_num,
#                         "line": i,
#                         "text": line.strip()
#                     })
#     return docs


# # ---- Step 2: Chunk lines semantically with line ranges ----
# def build_chunks(line_docs, chunk_size=800, chunk_overlap=100):
#     chunks = []
#     buffer, start_line, end_line, cur_page = "", None, None, None

#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size, chunk_overlap=chunk_overlap, separators=["\n", ". ", " "]
#     )

#     def make_citation(page, start, end):
#         if start == end:
#             return f"(p.{page}, line {start})"
#         return f"(p.{page}, lines {start}-{end})"

#     for entry in line_docs:
#         text, page, line = entry["text"], entry["page"], entry["line"]

#         if not buffer:
#             start_line, cur_page = line, page

#         buffer += text + " "
#         end_line = line

#         if len(buffer) > chunk_size:
#             start, end = min(start_line, end_line), max(start_line, end_line)
#             citation = make_citation(cur_page, start, end)
#             for split in splitter.split_text(buffer):
#                 chunks.append(
#                     Document(
#                         page_content=split,
#                         metadata={
#                             "page": cur_page,
#                             "line_start": start,
#                             "line_end": end,
#                             "citation": citation,
#                         },
#                     )
#                 )
#             buffer, start_line, end_line = "", None, None

#     if buffer:
#         start, end = min(start_line, end_line), max(start_line, end_line)
#         citation = make_citation(cur_page, start, end)
#         chunks.append(
#             Document(
#                 page_content=buffer,
#                 metadata={
#                     "page": cur_page,
#                     "line_start": start,
#                     "line_end": end,
#                     "citation": citation,
#                 },
#             )
#         )

#     return chunks


# # ---- Step 3: Build vectorstore ----
# def build_vectorstore(docs):
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     return FAISS.from_documents(docs, embeddings)


# # ---- Step 4: Cross-Encoder Re-Ranking ----
# reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# def rerank_docs(query, docs, top_n=8):
#     pairs = [(query, d.page_content) for d in docs]
#     scores = reranker.predict(pairs)
#     for doc, score in zip(docs, scores):
#         doc.metadata["rerank_score"] = float(score)
#     ranked = sorted(docs, key=lambda d: d.metadata["rerank_score"], reverse=True)
#     return ranked[:top_n]


# # ---- Step 5: Retrieval chain with reranking ----
# def build_retrieval_chain(db):
#     prompt = ChatPromptTemplate.from_template("""
# You are a legal analyst. Using ONLY the provided context, produce EXACTLY 10 pivotal items for the case.

# Structure:
# - Concise Title
# - Arguments (FOR or AGAINST, based only on the context)
# - Citations (precise page and line numbers)

# Rules:
# - Provide 5 FOR and 5 AGAINST items (total 10).
# - Do not mix both sides within one item.
# - Use only context. If implied, note it and cite the closest support.

# <context>
# {context}
# </context>

# Question: {input}
# """)

#     document_prompt = PromptTemplate.from_template(
#         "{page_content}\n[Citation: {citation}]"
#     )

#     llm = GroqLLM(
#         model="llama-3.3-70b-versatile",
#         temperature=0.1,
#         max_tokens=2048
#     )

#     document_chain = create_stuff_documents_chain(
#         llm=llm,
#         prompt=prompt,
#         document_variable_name="context",
#         document_prompt=document_prompt,
#     )

#     retriever = db.as_retriever(search_kwargs={"k": 20})  # pull more, reranker will prune

#     def custom_retrieval(query: str):
#         docs = retriever.get_relevant_documents(query)
#         ranked_docs = rerank_docs(query, docs, top_n=8)
#         return document_chain.invoke({"input": query, "context": ranked_docs})

#     return custom_retrieval


# # ---- Main Run ----
# if __name__ == "__main__":
#     pdf_path = "doc.pdf"

#     print("📖 Extracting PDF lines...")
#     line_docs = extract_lines_from_pdf(pdf_path)

#     print("🔗 Building semantic chunks...")
#     chunks = build_chunks(line_docs, chunk_size=800, chunk_overlap=100)

#     print("📚 Building vectorstore...")
#     db = build_vectorstore(chunks)

#     print("⚖️ Running retrieval chain with reranker...")
#     retrieval_chain = build_retrieval_chain(db)

#     query = "Identify the top 10 key items that are pivotal to the case."
#     response = retrieval_chain(query)   # note: now it's a callable

#     print("\nGroq-powered response:\n")
#     print(response)


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

c:\Users\prani\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prani\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not install

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

📖 Extracting PDF lines...
🔗 Building semantic chunks...
📚 Building vectorstore...
⚖️ Running retrieval chain with reranker...


C:\Users\prani\AppData\Local\Temp\ipykernel_64020\2616836860.py:147: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



Groq-powered response:

Here are the 10 pivotal items for the case, with 5 items FOR and 5 items AGAINST the FDA's actions:

1. **FOR**: The public interest and equities support injunctive relief against the FDA's actions on Mifepristone, as they defy federal law (p. 7, lines 1-5).
2. **AGAINST**: The FDA's actions are supported by regulations, including 21 C.F.R. § 314.2 and 21 C.F.R. § 314.500 (p. 13-14, lines 1-10).
3. **FOR**: The FDA's actions harm the public interest by undermining states' ability to protect their citizens and forcing states to divert scarce resources (p. 14, lines 1-10).
4. **AGAINST**: The Supreme Court has recognized the responsibility of federal agencies in regulating medications, as seen in Gonzales v. Oregon (p. 12, lines 1-10).
5. **FOR**: The FDA's actions are in conflict with the Constitutional provision of U.S. Const. amend. X, which reserves powers to the states (p. 14, lines 1-5).
6. **AGAINST**: The FDA's actions are supported by statutory authority

In [4]:
import pdfplumber
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from sentence_transformers import CrossEncoder
from groq import Groq  # assuming you have this imported elsewhere


# ---- Enhanced Step 1: Extract with better legal context ----
def extract_lines_from_pdf(pdf_path):
    docs = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            if not page.extract_text():
                continue
            lines = page.extract_text().split("\n")
            for i, line in enumerate(lines, start=1):
                if line.strip():
                    docs.append({
                        "page": page_num,
                        "line": i,
                        "text": line.strip()
                    })
    return docs


# ---- Enhanced Step 2: Legal-aware chunking ----
def build_legal_chunks(line_docs, chunk_size=1200, chunk_overlap=200):
    """Enhanced chunking that preserves legal argument structure"""
    chunks = []
    buffer, start_line, end_line, cur_page = "", None, None, None

    # Legal-specific separators that preserve argument structure
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap, 
        separators=["\n\n", "\n", ". ", "; ", ", ", " "],
        keep_separator=True
    )

    def make_citation(page, start, end):
        if start == end:
            return f"(p.{page}, line {start})"
        return f"(p.{page}, lines {start}-{end})"

    for entry in line_docs:
        text, page, line = entry["text"], entry["page"], entry["line"]

        if not buffer:
            start_line, cur_page = line, page

        buffer += text + " "
        end_line = line

        # Larger chunks for better legal context
        if len(buffer) > chunk_size:
            start, end = min(start_line, end_line), max(start_line, end_line)
            citation = make_citation(cur_page, start, end)
            for split in splitter.split_text(buffer):
                if len(split.strip()) > 100:  # Filter out tiny fragments
                    chunks.append(
                        Document(
                            page_content=split,
                            metadata={
                                "page": cur_page,
                                "line_start": start,
                                "line_end": end,
                                "citation": citation,
                            },
                        )
                    )
            buffer, start_line, end_line = "", None, None

    if buffer and len(buffer.strip()) > 100:
        start, end = min(start_line, end_line), max(start_line, end_line)
        citation = make_citation(cur_page, start, end)
        chunks.append(
            Document(
                page_content=buffer,
                metadata={
                    "page": cur_page,
                    "line_start": start,
                    "line_end": end,
                    "citation": citation,
                },
            )
        )

    return chunks


# ---- Enhanced Step 3: Multiple retrieval strategies ----
def build_vectorstore(docs):
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    return FAISS.from_documents(docs, embeddings)


# ---- Enhanced Step 4: Legal-focused reranking ----
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def legal_rerank_docs(query, docs, top_n=12):
    """Enhanced reranking with legal relevance scoring"""
    pairs = [(query, d.page_content) for d in docs]
    scores = reranker.predict(pairs)
    
    # Boost scores for legal keywords
    legal_keywords = [
        'FDA', 'mifepristone', 'Subpart H', 'REMS', 'regulation', 'statute',
        'U.S.C.', 'C.F.R.', 'violation', 'approval', 'safety', 'Dobbs',
        'constitutional', 'preemption', 'state law', 'federal law',
        'preliminary injunction', 'public interest'
    ]
    
    for doc, score in zip(docs, scores):
        # Boost score if document contains multiple legal keywords
        keyword_count = sum(1 for keyword in legal_keywords 
                          if keyword.lower() in doc.page_content.lower())
        boost = min(keyword_count * 0.1, 0.5)  # Cap boost at 0.5
        doc.metadata["rerank_score"] = float(score + boost)
        doc.metadata["keyword_count"] = keyword_count
    
    ranked = sorted(docs, key=lambda d: d.metadata["rerank_score"], reverse=True)
    return ranked[:top_n]


# ---- Enhanced Step 5: Legal-specific retrieval with multi-query approach ----
def build_enhanced_retrieval_chain(db):
    # Enhanced prompt with legal expertise and specific structure
    prompt = ChatPromptTemplate.from_template("""
You are an expert legal analyst specializing in administrative law, constitutional law, and federal court litigation. 

Your task: Analyze the provided legal document context to identify EXACTLY 10 pivotal items for this case, ensuring balanced representation of both sides.

REQUIRED OUTPUT STRUCTURE:
For each item, provide:
1. **Descriptive Title** (specific legal issue, not generic)
2. **Position**: FOR Plaintiffs/States OR AGAINST (FDA/Federal position)
3. **Legal Basis**: Specific statute, regulation, or constitutional provision
4. **Strategic Importance**: Why this matters for the case outcome
5. **Citation**: Precise page and line references from context

ANALYSIS REQUIREMENTS:
- Identify 5 items supporting plaintiffs/amici states
- Identify 5 items supporting defendants/federal government
- Focus on specific legal theories, not general principles
- Prioritize concrete statutory/regulatory violations over abstract concepts
- Include both procedural and substantive legal arguments

LEGAL FOCUS AREAS TO EXAMINE:
- Regulatory authority and statutory interpretation
- Constitutional federalism and state sovereignty
- Administrative law compliance
- Specific statutory violations (criminal, civil)
- Public interest and equitable considerations
- Standing and injury requirements

<context>
{context}
</context>

Question: {input}

Provide your analysis in the structured format above, ensuring each item is legally precise and strategically relevant.
""")

    document_prompt = PromptTemplate.from_template(
        "Legal Text: {page_content}\n[Source: {citation}]"
    )

    # Use your existing LLM setup
    llm = GroqLLM(
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_tokens=3000  # Increased for detailed legal analysis
    )

    document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt,
        document_variable_name="context",
        document_prompt=document_prompt,
    )

    # Multi-query retrieval strategy
    def enhanced_retrieval(query: str):
        # Generate multiple focused queries for better coverage
        focused_queries = [
            "FDA regulatory authority Subpart H mifepristone approval violations",
            "federal criminal statutes 18 U.S.C. 1461 1462 abortion drugs mailing",
            "Dobbs decision state sovereignty abortion regulation authority",
            "FDA safety requirements REMS program abandonment risks",
            "state laws enforcement resources harm standing injury",
            "constitutional preemption federal state authority balance",
            "administrative law compliance statutory interpretation",
            "public interest preliminary injunction equitable factors"
        ]
        
        all_docs = []
        retriever = db.as_retriever(search_kwargs={"k": 30})
        
        # Retrieve for main query
        main_docs = retriever.get_relevant_documents(query)
        all_docs.extend(main_docs)
        
        # Retrieve for focused queries
        for fq in focused_queries[:4]:  # Limit to avoid too many docs
            focused_docs = retriever.get_relevant_documents(fq)
            all_docs.extend(focused_docs[:5])  # Take top 5 from each
        
        # Remove duplicates while preserving order
        seen = set()
        unique_docs = []
        for doc in all_docs:
            doc_id = (doc.metadata.get("page", 0), doc.metadata.get("line_start", 0))
            if doc_id not in seen:
                seen.add(doc_id)
                unique_docs.append(doc)
        
        # Enhanced reranking
        ranked_docs = legal_rerank_docs(query, unique_docs, top_n=12)
        
        return document_chain.invoke({"input": query, "context": ranked_docs})

    return enhanced_retrieval


# ---- Main execution with enhancements ----
if __name__ == "__main__":
    pdf_path = "doc.pdf"

    print("Extracting PDF with legal structure preservation...")
    line_docs = extract_lines_from_pdf(pdf_path)

    print("Building legal-aware semantic chunks...")
    chunks = build_legal_chunks(line_docs, chunk_size=1200, chunk_overlap=200)

    print("Building enhanced vectorstore...")
    db = build_vectorstore(chunks)

    print("Running enhanced legal retrieval chain...")
    retrieval_chain = build_enhanced_retrieval_chain(db)

    # Enhanced query with legal specificity
    query = """Identify the top 10 key legal items that are pivotal to this case, focusing on:
    1. Specific statutory and regulatory violations
    2. Constitutional federalism arguments  
    3. Administrative law compliance issues
    4. Concrete harms and standing requirements
    5. Both plaintiffs' and defendants' strongest legal positions"""
    
    response = retrieval_chain(query)

    print("\nEnhanced Legal Analysis:\n")
    print(response)

W0922 21:39:03.111000 23420 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Extracting PDF with legal structure preservation...
Building legal-aware semantic chunks...
Building enhanced vectorstore...


C:\Users\prani\AppData\Local\Temp\ipykernel_23420\2608013184.py:97: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Running enhanced legal retrieval chain...


C:\Users\prani\AppData\Local\Temp\ipykernel_23420\2608013184.py:208: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  main_docs = retriever.get_relevant_documents(query)



Enhanced Legal Analysis:

Here are the top 10 pivotal legal items in the case, with 5 supporting the plaintiffs/amici states and 5 supporting the defendants/federal government:

**1. Violation of 18 U.S.C. § 1461**
* **Descriptive Title**: Federal Law Prohibiting Mailing of Abortion-Inducing Drugs
* **Position**: FOR Plaintiffs/States
* **Legal Basis**: 18 U.S.C. § 1461
* **Strategic Importance**: This statute directly prohibits the mailing of abortion-inducing drugs, which is a key aspect of the case.
* **Citation**: (p.16, lines 6-10)

**2. States' Sovereign Interest in Regulating Abortion**
* **Descriptive Title**: Constitutional Authority of States to Regulate Abortion
* **Position**: FOR Plaintiffs/States
* **Legal Basis**: U.S. Const. amend. X
* **Strategic Importance**: This argument highlights the states' primary authority to regulate abortion, which is a crucial aspect of the case.
* **Citation**: (p.18, lines 5-10)

**3. FDA's Defiance of Federal Criminal Law**
* **Descripti